In [1]:
import pandas as pd
import numpy as np
import pyspark

### Ejercicio 1
1) Se cuenta con un RDD con información sobre patentamientos
de autos con la siguiente información (patente, marca, modelo,
versión, tipo_vehiculo, provincia, fecha), donde tipo_vehiculo
indica si la unidad patentada es auto, pickup, camión o moto.

Se pide generar un programa en pySpark que indique la marca y
modelo del auto más patentado por tipo de vehículo en CABA en el mes de Abril de 2017. (***) (15 pts)

In [26]:
#(patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha)
patentamientos = [
    ("1",  "FORD",   "FIESTA", "BASE", "AUTO", "CABA",       "2017-04-18"),
    ("2",  "FORD",   "FOCUS",  "BASE", "AUTO", "CABA",       "2017-04-20"),
    ("3",  "FORD",   "FIESTA", "FULL", "AUTO", "MISIONES",   "2017-04-22"),
    ("4",  "FORD",   "FIESTA", "BASE", "AUTO", "CORRIENTES", "2017-04-14"),
    ("5",  "FORD",   "FIESTA", "BASE", "AUTO", "CABA",       "2017-05-27"),
    ("6",  "FORD",   "FIESTA", "BASE", "AUTO", "CABA",       "2017-04-10"),
    ("7",  "FORD",   "FOCUS",  "BASE", "AUTO", "CORDOBA",    "2017-04-04"),
    ("8",  "FORD",   "FIESTA", "FULL", "AUTO", "MISIONES",   "2017-04-16"),
    ("9",  "FORD",   "FIESTA", "BASE", "AUTO", "CORDOBA",    "2017-04-15"),
    ("10", "FORD",   "FOCUS",  "BASE", "AUTO", "CABA",       "2017-05-26"),
    ("11", "YAMAHA", "XR01",   "BASE", "MOTO", "CABA",       "2017-04-18"),
    ("12", "YAMAHA", "XR01",   "BASE", "MOTO", "CABA",       "2017-04-20"),
    ("13", "YAMAHA", "XR01",   "FULL", "MOTO", "MISIONES",   "2017-04-22"),
    ("14", "YAMAHA", "XR02",   "BASE", "MOTO", "CORRIENTES", "2017-04-14"),
    ("15", "YAMAHA", "XR02",   "BASE", "MOTO", "CABA",       "2017-05-27"),
    ("16", "YAMAHA", "XR02",   "BASE", "MOTO", "CABA",       "2017-04-10"),
    ("17", "YAMAHA", "XR02",   "BASE", "MOTO", "CORDOBA",    "2017-04-04"),
    ("18", "YAMAHA", "XR02",   "FULL", "MOTO", "MISIONES",   "2017-04-16"),
    ("19", "YAMAHA", "XR02",   "BASE", "MOTO", "CORDOBA",    "2017-04-15"),
    ("20", "YAMAHA", "XR01",   "BASE", "MOTO", "CABA",       "2017-05-26"),
    ("21", "HONDA",  "CV01",   "BASE", "MOTO", "CABA",       "2017-04-18"),
    ("22", "HONDA",  "CV01",   "BASE", "MOTO", "CABA",       "2017-04-02"),
    ("23", "HONDA",  "CV01",   "FULL", "MOTO", "MISIONES",   "2017-04-25"),
    ("24", "HONDA",  "CV02",   "BASE", "MOTO", "FORMOSA",    "2017-04-14"),
    ("25", "HONDA",  "CV02",   "BASE", "MOTO", "CABA",       "2017-05-27"),
    ("26", "HONDA",  "CV02",   "BASE", "MOTO", "CABA",       "2017-04-12"),
    ("27", "HONDA",  "CV02",   "BASE", "MOTO", "CORDOBA",    "2017-04-04"),
    ("28", "HONDA",  "CV02",   "FULL", "MOTO", "MISIONES",   "2017-04-01"),
    ("29", "HONDA",  "CV02",   "BASE", "MOTO", "CORDOBA",    "2017-04-15"),
    ("30", "HONDA",  "CV01",   "BASE", "MOTO", "CABA",       "2017-05-07"),
    ("31", "FIAT",   "UNO",    "BASE", "AUTO", "CABA",       "2017-04-18"),
    ("32", "FIAT",   "UNO",    "BASE", "AUTO", "CHUBUT",     "2017-04-02"),
    ("33", "FIAT",   "DOS",    "FULL", "AUTO", "MISIONES",   "2017-04-08"),
    ("34", "FIAT",   "UNO",    "BASE", "AUTO", "CORRIENTES", "2017-04-14"),
    ("35", "FIAT",   "UNO",    "FULL", "AUTO", "CABA",       "2017-05-09"),
    ("36", "FIAT",   "DOS",    "BASE", "AUTO", "CHUBUT",     "2017-04-10"),
    ("37", "FIAT",   "UNO",    "BASE", "AUTO", "CORDOBA",    "2017-04-04"),
    ("38", "FIAT",   "UNO",    "FULL", "AUTO", "FORMOSA",    "2017-04-01"),
    ("39", "FIAT",   "DOS",    "BASE", "AUTO", "CORDOBA",    "2017-04-15"),
]

rdd = sc.parallelize(patentamientos)
print("Cantidad de patentamientos: {}".format(rdd.count()))
rdd1 = rdd.filter(lambda x: (x[6][:7]=='2017-04' and x[5]=='CABA'))
print("Cantidad de patentamientos durante Abril 2017 en CABA: {}".format(rdd1.count()))
rdd2 = rdd1.map(lambda x: ("{}-{}-{}".format(x[4],x[1],x[2]),1)) #("TIPO-MARCA-MODELO",1)
print("('TIPO-MARCA-MODELO',1):\n")
print(rdd2.collect())
rdd3 = rdd2.reduceByKey(lambda x,y: x+y) #("TIPO-MARCA-MODELO",cantidad)
print("('TIPO-MARCA-MODELO',cantidad):\n")
print(rdd3.collect())   
def  separar(elemento_rdd3):
    tipo, marca, modelo = elemento_rdd3[0].split("-")
    return (tipo,("{}-{}".format(marca,modelo),elemento_rdd3[1]))
rdd4 = rdd3.map(separar) #(tipo,(marca-modelo,cantidad))
print("(tipo,(marca-modelo,cantidad)):\n")
print(rdd4.collect())
rdd5 = rdd4.reduceByKey(lambda x,y: x if (x[1] > y[1]) else y) #me quedo con el máximo
rdd6 = rdd5.map(lambda x: "{}: {}".format(x[0],x[1][0].replace("-"," ")))
rdd6.collect()

Cantidad de patentamientos: 39
Cantidad de patentamientos durante Abril 2017 en CABA: 10
('TIPO-MARCA-MODELO',1):

[('AUTO-FORD-FIESTA', 1), ('AUTO-FORD-FOCUS', 1), ('AUTO-FORD-FIESTA', 1), ('MOTO-YAMAHA-XR01', 1), ('MOTO-YAMAHA-XR01', 1), ('MOTO-YAMAHA-XR02', 1), ('MOTO-HONDA-CV01', 1), ('MOTO-HONDA-CV01', 1), ('MOTO-HONDA-CV02', 1), ('AUTO-FIAT-UNO', 1)]
('TIPO-MARCA-MODELO',cantidad):

[('MOTO-YAMAHA-XR02', 1), ('MOTO-YAMAHA-XR01', 2), ('AUTO-FORD-FIESTA', 2), ('MOTO-HONDA-CV01', 2), ('AUTO-FIAT-UNO', 1), ('AUTO-FORD-FOCUS', 1), ('MOTO-HONDA-CV02', 1)]
(tipo,(marca-modelo,cantidad)):

[('MOTO', ('YAMAHA-XR02', 1)), ('MOTO', ('YAMAHA-XR01', 2)), ('AUTO', ('FORD-FIESTA', 2)), ('MOTO', ('HONDA-CV01', 2)), ('AUTO', ('FIAT-UNO', 1)), ('AUTO', ('FORD-FOCUS', 1)), ('MOTO', ('HONDA-CV02', 1))]


['MOTO: HONDA CV01', 'AUTO: FORD FIESTA']

### Ejercicio 2
2) Tenemos un dataframe con la información de distintas playlists
armadas por usuarios con el formato (playlist, song_id,
description). A su vez, contamos con un dataframe de canciones
que contiene (song_id, singer, year, lenght, genres).

Se pide generar un programa en Pandas que indique para cada
playlist cual es el cantante predominante (con mas canciones
incluidas dentro de esa lista). (**) (15 pts)

In [74]:
columns_playlists = ("playlist", "song_id")
columns_songs = ("song_id", "singer")
playlists = [
    ("tranqui","acuestate"),
    ("tranqui","relajate"),
    ("tranqui","duerme mejor"),
    ("tranqui","bohemia"),
    ("tranqui","rompan todo"),
    ("tranqui","todos chorros"),
    ("punk","rompan todo"),
    ("punk","todos chorros"),
    ("punk","movete"),
    ("punk","activismo"),
    ("punk","rojo"),
    ("punk","acuestate"),
    ("punk","dulce como vos"),
    ("punk","estrella de mar"),
    ("pop","bailalo bien"),
    ("pop","dulce como vos"),
    ("pop","estrella de mar"),
    ("pop","duerme mejor"),
]
songs = [
    ("acuestate", "Franco"),
    ("relajate", "Axel"),
    ("mejor", "Franco"),
    ("bohemia", "Axel"),
    ("rompan todo", "Axel"),
    ("todos chorros", "Laurita"),
    ("movete", "Lucia"),
    ("activismo", "Franco"),
    ("rojo", "Axel"),
    ("dulce como vos", "Laurita"),
    ("bailalo bien", "Marina"),
    ("estrella de mar", "Franco"),
    ("duerme mejor", "Marina"),
]
df_songs = pd.DataFrame(data=songs, columns=columns_songs)
df_playlists = pd.DataFrame(data=playlists, columns=columns_playlists)

df_songs.index = df_songs.song_id
df_playlists['singer'] = df_playlists.song_id.map(df_songs.singer)
temp = df_playlists.groupby(["playlist","singer"]).agg({"singer":"count"}).unstack(fill_value=0)
temp.columns = temp.columns.levels[1]
temp = temp.idxmax(axis=1)
df = pd.DataFrame(data=t.values,columns=["Top Artist"],index=t.index)
df

,Top Artist
playlist,
pop,Marina
punk,Franco
tranqui,Axel


### Ejercicio 3
b) Es imposible que un archivo
comprimido mediante huffman
estático de orden 1 iguale la
máxima compresión dada por la
entropía del mismo. (***) (10
pts)
Verdadero. Tiene que guardar una tabla de frecuencias
D ) Tenemos 2 archivos, uno con
longitud pequeña y el otro muy
grande que se comprimen
utilizando huffman estático de
orden 5. Si observamos que
tienen la mismas tablas de
frecuencias podemos afirmar que
el cociente entre el tamaño del
archivo sin comprimir y el
tamaño del archivo comprimido
será similar. (***) (10 pts)

Falso. Si tienen la misma tabla de frecuencias, el archivo más grande se comprimió mejor

### Ejercicio 5
5) Descomprimir el siguiente archivo comprimido con LZ78/LZW:
B C 256 258 B A 260 262 260

Indicar cuanto se logro comprimir el archivo en este caso.
(***) (15 pts)

In [ ]:
#B C 256 258 B A 260 262 260 -> 9*9= 81 bytes (88 porque tengo que poner 7 de relleno -> compresión = 31.25% )
1)B-256:BC
2)C-257:CB
3)BC(256)-258:BCB
4)BCB(258)-259:BCBB
5)B-260:BA
6)A-261:AB
7)BA(260)-262:BAB
8)BAB(262)-263:BABB
9)BA
-> B C B C B C B B A B A B A B B A -> 8*16=128 bytes
Verifico:
1)B-256:BC
2)C-257:CB
3)BC(256)-258:BCB
4)BCB(258)-259:BCBB     
5)B-260:BA
6)A-261:AB
7)BA(260)-262:BAB
8)BAB(262)-263:BABB
9)A(260)
#B C 256 258 B A 260 262 260

### Ejercicio 7
7) Tenemos un total de 10.000 claves de 32 bytes c/u. Si usamos el
esquema FKS y la primer tabla tiene 1000 posiciones. ¿Cuánto
espacio necesitamos en total para almacenar las 10.000 claves?
(10 pts *)

In [ ]:
2 x m = 2 x 10.0000 = 20.000
20.000 x 32 = 640.000 bytes